In [69]:
import praw
import sqlite3

conn = sqlite3.connect('reddit.db')
c = conn.cursor()

c.execute('''DROP TABLE IF EXISTS topics''')
c.execute('''DROP TABLE IF EXISTS comments''')

c.execute('''CREATE TABLE topics (topicTitle text, topicText text, topicID text, topicCategory text)''')
c.execute('''CREATE TABLE comments (commentText text, commentID text, topicTitle text, topicText text, topicID text, topicCategory text)''')

user_agent = "Introduceing Data Science Book"
r = praw.Reddit(user_agent=user_agent, client_id = '4D2A7pe9rUVauw', client_secret = 'fzm6SEcsjh9DJN3_vzmKTtnaBPM')

subreddits = ['datascience', 'gameofthrones']

limit = 1000



In [70]:
def prawGetData(limit, subredditName):
    topics = r.subreddit(subredditName).hot(limit = limit)
    print(f'processing {subredditName}')
    commentInsert = []
    topicInsert = []
    topicNBR = 1
    for topic in topics:
        if (topicNBR % 100==0):
            print(f'topic no {topicNBR}')
        topicNBR += 1
        try:
            topicInsert.append((topic.title, topic.selftext, topic.id, subredditName))
        except Exception as e:
            print(e)
            pass
        try:
            for comment in topic.comments:
                commentInsert.append((comment.body, comment.id, topic.title, topic.selftext, topic.id, subredditName))
        except Exception as e:
            print(e)
            pass
    
    # print(topicInsert)
    # print(commentInsert)

    print ('inserting data into sql lite')

    c.executemany('INSERT INTO topics VALUES (?,?,?,?)', topicInsert)
    c.executemany('INSERT INTO comments VALUES (?,?,?,?,?,?)', commentInsert)

    conn.commit()


In [71]:
c = conn.cursor()
c.execute('SELECT * FROM topics')
x = c.fetchone()
print(x)

None


In [72]:
for subject in subreddits:
    prawGetData(limit = limit, subredditName=subject)

processing datascience
'Submission' object has no attribute 'selfText'
'Submission' object has no attribute 'selfText'
'Submission' object has no attribute 'selfText'
'Submission' object has no attribute 'selfText'
'Submission' object has no attribute 'selfText'
'Submission' object has no attribute 'selfText'
'Submission' object has no attribute 'selfText'
'Submission' object has no attribute 'selfText'
'Submission' object has no attribute 'selfText'
'Submission' object has no attribute 'selfText'
'Submission' object has no attribute 'selfText'
'Submission' object has no attribute 'selfText'
'Submission' object has no attribute 'selfText'
'Submission' object has no attribute 'selfText'
'Submission' object has no attribute 'selfText'


KeyboardInterrupt: 

# Data preparation

In [55]:
import nltk
import matplotlib.pyplot as plt
from collections import OrderedDict
import random

nltk.download('punkt')
nltk.download('stopwords')
#conn = sqlite3.connect('reddit.db')
#c = conn.cursor()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\E10112538\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\E10112538\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [56]:
def wordFilter(excluded, wordrow):
    filtered = [word for word in wordrow if word not in excluded]
    return filtered

stopwords = nltk.corpus.stopwords.words('english')

def lowerCaseArray(wordrow):
    lowercased = [word.lower() for word in wordrow]
    return lowercased

In [57]:
print(stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [58]:
def data_processing(sql):
    c.execute(sql)
    data = {'wordMatrix': [], 'all_words': []}
    row = c.fetchone()
    print(row)
    while row is not None:
        wordrow = nltk.tokenize.word_tokenize(row[0] + " " + row[1])
        wordrow_lowercased = lowerCaseArray(wordrow)
        wordrow_nostopwords = wordFilter(stopwords, wordrow_lowercased)
        datap['all_words'].extend(wordrow_nostopwords)
        datap['wordMatrix'].append(wordrow_nostopwords)
        row = c.fetchone()
    return data

subreddits = ['datascience','gameofthrones']
data = {}
for subject in subreddits:
    data[subject] = data_processing(sql=f"SELECT topicTitle, topicText, topicCategory FROM topics WHERE topicCategory = '{subject}'")

None
None


In [51]:
c.execute("SELECT count() FROM topics")
x = c.fetchone()
print(x)

(0,)
